In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Dropout, Flatten
import matplotlib.pyplot as plt
import numpy as np
import scipy
%matplotlib inline

In [ ]:
import glob
# import imageio
from skimage import color
from skimage import io
from skimage.transform import resize

fruit_types = glob.glob('../datasets/fruits-360/Training/*')
fruit_types.sort
fruit_types
X_train = []
y_train = []
for i in range(len(fruit_types)):
    fruit_images = glob.glob(fruit_types[i] + '/*')
    for image in fruit_images:
        gray_image = color.rgb2gray(io.imread(image))
        resize_image = resize(gray_image, (gray_image.shape[0] / 4, gray_image.shape[1] / 4), mode='constant')
        X_train += [resize_image]
        y_train.append(i)
plt.imshow(X_train[0])

In [ ]:
test_types = glob.glob('../datasets/fruits-360/Validation/*')
test_types.sort
test_types
X_test = []
y_test = []
for i in range(len(test_types)):
    test_images = glob.glob(test_types[i] + '/*')
    for image in test_images:
        gray_image = color.rgb2gray(io.imread(image))
        resize_image = resize(gray_image, (gray_image.shape[0] / 4, gray_image.shape[1] / 4), mode='constant')
        X_test += [resize_image]
        y_test.append(i)
plt.imshow(X_test[0])

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)

print("Shape of X_train: {}".format(X_train.shape))
print("Shape of X_test: {}".format(X_test.shape))

In [ ]:
X_train = X_train.reshape(29228, 625).astype('float32')
X_test = X_test.reshape(9837, 625).astype('float32')

In [ ]:
X_test[0]

In [ ]:
# X_train /= 255
# X_test /= 255

In [ ]:
y_train = keras.utils.to_categorical(y_train, 60)
y_test = keras.utils.to_categorical(y_test, 60)

In [ ]:
model = Sequential()
model.add(Dense(625, activation='tanh', input_shape=(625,)))
model.add(Dense(300, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(60, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

In [ ]:
# model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1, validation_data=(X_test, y_test))

In [ ]:
X_train = X_train.reshape(29228, 25, 25, 1).astype('float32')
X_test = X_test.reshape(9837, 25, 25, 1).astype('float32')


In [ ]:
X_train.shape

In [ ]:
cn_model = Sequential()
cn_model.add(Conv2D(25, (3, 3), padding='same', input_shape=X_train.shape[1:]))
cn_model.add(Activation('relu'))
cn_model.add(Conv2D(25, (3, 3), padding='same'))
cn_model.add(Activation('relu'))
cn_model.add(MaxPooling2D(pool_size=(2, 2)))
cn_model.add(Dropout(0.25))

cn_model.add(Conv2D(25, (3, 3), padding='same'))
cn_model.add(Activation('relu'))
cn_model.add(Conv2D(32, (3, 3), padding='same'))
cn_model.add(Activation('relu'))
cn_model.add(MaxPooling2D(pool_size=(2, 2)))
cn_model.add(Dropout(0.25))

cn_model.add(Flatten())
cn_model.add(Dense(512, activation='relu'))
cn_model.add(Dropout(0.5))
cn_model.add(Dense(256, activation='relu'))
cn_model.add(Dropout(0.5))
cn_model.add(Dense(128, activation='relu'))
cn_model.add(Dropout(0.5))
cn_model.add(Dense(60, activation='softmax'))

cn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

cn_model.summary()

In [ ]:
cn_model.fit(X_train, y_train, batch_size=128, epochs=5, validation_data=(X_test, y_test), shuffle=True)

In [ ]:
# What is the baseline?
# Coin flip? pick the most common option? / what is the ditribution?